# Replication: CS-Flow MVTec AD Cable, rotated

In this notebook, we rotate the training data for the MVTec AD category *cable*.

25% of all data is each 0°, 90°, 180°, and 270° rotated.

In [1]:
from __init__ import activate_cs_flow_config, CSFLOW_DIR
import shutil

# First, we activate the correct configuration.
activate_cs_flow_config(name='mvtec-cable')

# Extracting Features

In [2]:
import os
import numpy as np
from src.repos.cs_flow.utils import load_datasets, make_dataloaders
from src.repos.cs_flow.extract_features import extract
import src.repos.cs_flow.config as c


prepend_dir = str(CSFLOW_DIR.resolve()) + '/'
export_name = c.class_name
export_dir = prepend_dir + 'data/features/' + export_name + '/'
image_paths = export_dir + c.class_name + '_image_paths.npy'
c.pre_extracted = False
os.makedirs(export_dir, exist_ok=True)
train_set, test_set = load_datasets(prepend_dir + c.dataset_path, c.class_name)
train_loader, test_loader = make_dataloaders(train_set, test_set)

In [3]:
from pathlib import Path

if not Path(image_paths).exists():
    extract(train_loader, test_loader, export_dir, c.class_name)
    paths = [p for p, l in test_set.samples]
    np.save(export_dir + c.class_name + '_image_paths.npy', paths)

Loaded pretrained weights for efficientnet-b5


100%|██████████| 10/10 [00:09<00:00,  1.04it/s]


# Training

In [ ]:
from src.repos.cs_flow.train import train

results = train(train_loader, test_loader)
if isinstance(results, tuple):
    model = results[0]
    print(results[1:])
else:
    model = results

Loaded pretrained weights for efficientnet-b5

Train epoch 0


100%|██████████| 14/14 [00:16<00:00,  1.16s/it]


Epoch: 0.0 	 train loss: 14.8853


100%|██████████| 14/14 [00:15<00:00,  1.14s/it]


Epoch: 0.4 	 train loss: 7.4750


...

100%|██████████| 14/14 [00:15<00:00,  1.14s/it]


Epoch: 0.16 	 train loss: 1.9157


100%|██████████| 14/14 [00:15<00:00,  1.12s/it]


Epoch: 0.20 	 train loss: 1.8783


 50%|█████     | 7/14 [00:07<00:07,  1.12s/it]

# Evaluation

In [5]:
localize = True
upscale_mode = 'bilinear'
score_export_dir = CSFLOW_DIR.joinpath('./viz/scores/', c.modelname)
os.makedirs(score_export_dir, exist_ok=True)
map_export_dir = CSFLOW_DIR.joinpath('./viz/maps/', c.modelname)
os.makedirs(map_export_dir, exist_ok=True)

In [ ]:
from src.repos.cs_flow.model import load_model
from src.repos.cs_flow.evaluate import evaluate

img_paths = test_set.paths if c.pre_extracted else [p for p, l in test_set.samples]
_, test_loader = make_dataloaders(train_set, test_set)
mod = load_model(c.modelname)
evaluate(mod, test_loader, score_export_dir, map_export_dir, img_paths, localize)